![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png)  ![Python Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png)
# Comptant paraules: Construeix una aplicació que compti paraules de forma eficient

En aquesta tasca s'utilitzarà Pyspark per desenvolupar una aplicació de comptatge de paraules.

Amb l'ús massiu d'Internet i les xarxes socials, el volum de text no estructurat esta creixent dramàticament, i Spark és una gran eina per analitzar aquest tipus de dades. En aquesta tasca, anem a escriure codi per trobar les paraules més comuns en un text generat en latin, el ja conegut [Lorem Ipsum](https://www.lipsum.com/).


El més interessant de la forma de treballar en aquesta tasca és que podria escalar-se a casos de big data com, per exemple, trobar les paraules més comuns a la Wikipedia.

## Durant aquesta TASCA cobrirem les següents parts:

* *Part 1:* Creació d'un RDD i un pair RDD
* *Part 2:* Comptar paraules fent servir un pair RDD
* *Part 3:* Trobar les paraules individuals i la seva freqüència d'aparició mitjana
* *Part 4:* Aplicar les funcionalitats desenvolupades a un arxiu de text *
* *Part 5:* Calcular alguns estadístics *


> Com a referència a tots els detalls dels mètodes que es fan servir en aquesta pràctica utilitzar:
> * [API Python de Spark](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD)

Per començar, cal carregar l'entorn. Com a nom d'app poseu **M3T01_nom_cognom**.

Recordeu afegir tots els comentaris o quadres de text necessaris per explicar detalladament tot el que es fa i justificar les decisions així com comentar els resultats.

<span style="color:navy"><strong>-> Per començar, resumim les etapes de descarrega i instal·lació del Spark per a que tot funcioni correctament</strong></span>

    * Verificar si es té Java 8 (1.8.231 en el meu cas) ; si no, escriure al Anaconda prompt o Notebook : 'conda install -c cyclus java-jdk'
    * Descarregar el fitxer per a Windows en 'https://www.apache.org/dyn/closer.lua/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz'
    * Posar el fitxer descarregat 'spark-3.5.1-bin-hadoop3.tgz' a la carpeta 'C:\spark' creada previament
    * Extreure amb 'right clic' o per consola bash
    * cmd : 'setx SPARK_HOME "C:\spark\spark-3.5.1-bin-hadoop3"'
    * cmd : 'setx PATH "%PATH%;%SPARK_HOME%\bin"'
    * Anacona prompt o Notebook : 'pip install pyspark'
    * Anaconda prompt : 'pip install findspark'
    * Restart Notebook kernel and kepp on working...

In [2]:
pip install pyspark

     -------------------------------------- 317.0/317.0 MB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 200.5/200.5 kB 5.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488515 sha256=23cd7cb3105e2d3b2fcdfde135feebbdb4a26245cdfaafd2282ce781f9de7619
  Stored in directory: c:\users\buba\appdata\local\pip\cache\wheels\92\09\11\aa01d01a7f005fda8a66ad71d2be7f8aa341bddafb27eee3c7
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


### (0) Configuració de l'entorn python + spark
Poseu nom a la aplicació (appName) en format "**M3T01_**+Nom_Cognom"

In [8]:
import findspark
findspark.init()
import pyspark
import random

# Check if SparkContext is already running
if 'sc' not in locals():
    sc = pyspark.SparkContext(master="local[1]", appName="M3T01_Cristina_Cosma")
else:
    # Stop the existing SparkContext
    sc.stop()
    
    # Create a new SparkContext
    sc = pyspark.SparkContext(master="local[1]", appName="M3T01_Cristina_Cosma")


<span style="color:navy"><strong>-> He canviat el codi inicial per un if-else, ja que al executar més d'un cop, la instància donava error<em>PySpark</em></strong></span>

    Aquest fragment de codi inicialitza un SparkContext anomenat sc. Si no hi ha cap SparkContext existent ('sc' no està a locals()), es crea un de nou amb la configuració especificada (master="local[1]" per executar Spark localment amb un únic fil de treball) i el nom de l'aplicació proporcionat. Si es troba un SparkContext existent, el deté i llavors en crea un de nou amb la mateixa configuració.

## Part 1: Creació d'un RDD i un pair RDDs

En aquesta secció, explorarem com crear RRDs usant `parallelize` i com aplicar pair RDDs al problema del recompte de paraules.

<span style="color:navy"><strong>-> Recordatori de conceptes</strong></span>: els RDDs proporcionen la base per al processament de dades distribuïdes a Apache Spark, mentre que els pair RDDs són específicament útils per a operacions que impliquen parells de clau-valor, com ara el recompte de paraules on necessitem agregar els comptes de paraules per claus (paraules en aquest cas). Aprofitant aquests conceptes, es pot implementar de manera eficient algorismes per processar i analitzar conjunts de dades a gran escala en paral·lel. Per implementar un algoritme per comptar paraules en un text donat utilitzant PySpark, recordem què són els RDD (Resilient Distributed Datasets) i els pair RDDs:

- <span style="color:navy"><strong>RDD (Resilient Distributed Dataset)</strong></span>:
El RDD és l'estructura de dades fonamental a Apache Spark, que representa una col·lecció distribuïda immutable d'objectes en els quals es poden operar en paral·lel.
Està particionat entre els nodes d'un clúster, permetent el processament paral·lel de dades.
Els RDDs suporten dos tipus d'operacions: transformacions i accions.
Les transformacions creen un nou RDD a partir d'un ja existent, com ara map, flatMap, filter, etc.
Les accions realitzen càlculs sobre un RDD i retornen els resultats al programa controlador, com ara collect, reduce, count, etc.
En el context del recompte de paraules, es pot crear un RDD a partir de les dades de text d'entrada i realitzar transformacions i accions per comptar les ocasions de cada paraula.
    
- <span style="color:navy"><strong>Pair RDD</strong></span>:
El pair RDD és un RDD on cada element és un parell de clau-valor, típicament utilitzat per operacions que requereixen agrupar o agregare les dades per claus.
Els pair RDDs permeten operacions més complexes com ara càlculs estil map-reduce.
En el context del recompte de paraules, es pot transformar el RDD de paraules en un pair RDD on la paraula és la clau i el valor és un comptador inicial (normalment 1).
Els pair RDDs suporten operacions específiques per parells de clau-valor, com ara reduceByKey, groupByKey, sortByKey, etc., que són útils per a l'agregació i la manipulació de dades basades en claus.

### Opcions de configuració PySpark
Les opcions que ens dona Spark en quant a optimitzar el rendiment son gairebé infinites, entre altres tenim la funció **setAll** que ens permet configurar el funcionament del framework al detall.

**Investiga sobre aquesta funció i destaca les opción que consideris més rellevants**

<span style="color:navy"><strong>-> Resposta sobre les opcions de configuració <em>PySpark</em></strong></span>

PySpark proporciona una àmplia gamma d'opcions de configuració per optimitzar el rendiment de les tasques de Spark. Una funció important per establir aquestes opcions és setAll(), que et permet configurar l'entorn d'execució de Spark amb detall. Vegem alguns exemples d'opcions de configuració que es poden establir utilitzant setAll(). Ajustant amb cura aquestes opcions en funció de la càrrega de treball específica, la mida de les dades i els recursos del clúster, es pot aconseguir un millor rendiment i utilització de recursos en les aplicacions de Spark.

Aquí teniu algunes de les opcions de configuració clau que podeu trobar rellevants:

- <strong>spark.executor.memory</strong>: Aquesta opció permet especificar la quantitat de memòria per assignar per executor. És crucial per controlar l'ús de memòria de les tasques de Spark i prevenir errors de falta de memòria.

- <strong>spark.executor.cores:</strong> Especifica el nombre de nuclis de CPU per assignar a cada executor. Ajustar aquesta opció pot optimitzar la utilització de recursos i la paral·lelisme.</strong>

- <strong>spark.default.parallelism</strong> Determina el nombre predeterminat de particions per a RDD, que afecta el paral·lelisme en operacions de Spark com map, reduce i join. Establir-lo adequadament en funció de la configuració del clúster i la mida de les dades pot millorar el rendiment de les tasques.</strong>

- <strong>spark.shuffle.service.enabled</strong> Activa o desactiva el servei de barreja extern, que gestiona la barreja de dades entre els executors. Habilitar aquesta opció pot millorar el rendiment de barreja i reduir la càrrega al node controlador.</strong>

- <strong>spark.sql.shuffle.partitions</strong> Especifica el nombre predeterminat de particions a utilitzar quan es barregen les dades en operacions de Spark SQL. Ajustar aquesta opció pot optimitzar el rendiment de les consultes SQL amb barreges.

In [23]:
# Veiem un exemple de com es crea i es verifica la creació d'una RDD

wordsList = ['cat', 'elephant', 'rat', 'rat', 'cat']

wordsRDD = sc.parallelize(wordsList)

# Print out the type of wordsRDD
type(wordsRDD)


pyspark.rdd.RDD

### (1b) Crear el plural de les paraules i testejar

Utilitzarem una transformació `map()` per incorporar la lletra 's' a cada un dels strings emmagatzemats en el RDD que acabem de crear. Anem a definir una funció de Python que retorni una paraula, que se li ha passat com paràmetre, incorporant una "s" al final de la mateixa. Substitueix el text `<FILL IN>` amb la solucio proposada. Després d'haver definit correctament la funció `makePlural`, executar la segona cel·la que conté un assert de test. Si la solució és correcta, s'imprimirà `1 test passed`.

Aquesta serà la forma habitual de treballar en les algunes de les tasques del curs. Els exercicis contindran una explicació del que s'espera, seguit d'una cel·la de codi amb un o més `<FILL IN>`. Les cel·les que necessitin ser modificades contindran el text `# TOT: Replace <FILL IN> with appropriate code` a la primera línia.

Un cop s'hagin substituït tots els `<FILL IN>` pel codi Python adequat, executar la cel·la, i posteriorment executar la cel·la següent de test per comprovar que que la solució és l'esperada.

<span style="color:navy"><strong>-> Reemplacem <'FILL IN'> amb *'word + 's'* com es defineix dins la mateixa celda als apartats Args i Note</strong></span>

In [19]:
# TODO: Replace <FILL IN> with appropriate code
def makePlural(word):
    """Adds an 's' to `word`.

    Note:
        This is a simple function that only adds an 's'.  

    Args:
        word (str): A string.

    Returns:
        str: A string with 's' added to it.
    """
    # return <FILL IN>
    return word + 's'    


makePlural('cat')

'cats'

In [24]:
# TEST Pluralize and test (1b)
assert makePlural('rat') == 'rats', 'incorrect result: makePlural does not add an s'

<span style="color:navy"><strong>-> Afegeixo una celda de comprobació perquè la anterior no imprimeix cap resultat (suposo que si no diu res és correcte, però en faig una altra per a més precisó</strong></span>

In [26]:
# TEST Pluralize and test (1b)

result = makePlural('rat')
expected_result = 'rats'

if result == expected_result:
    print("Test passed.")
else:
    print(f"incorrect result: makePlural does not add an 's', expected '{expected_result}' but got '{result}'")

Test passed.


### (1c) Aplicar `makePlural` al nostre RDD

Ara és el moment d'aplicar la nostra funció `makePlural()` a tots els elements del RDD usant una transformació [map()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.map). Posteriorment executar l'acció [collect ()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.collect) per obtenir el RDD transformat.

<span style="color:navy"><strong>-> Reemplacem <'FILL IN'> amb el nom de la funció que hem definit abans, en def *makePlural*(word)</strong></span>
    
* wordsRDD.map(makePlural) aplica la funció makePlural a cada element del RDD wordsRDD utilitzant la transformació map().pluralRDD.collect() 
* executa l'acció collect() en el RDD transformada pluralRDD, que recull tots els elements del RDD i els retorna com a llista.
* En executar aquest codi, sobté un nou RDD on cada paraula s'ha pluralitzat, i recollir el RDD ens donarà les dades transformades com a llista.

In [29]:
# TODO: Replace <FILL IN> with appropriate code

# pluralRDD = wordsRDD.map(<FILL IN>)
pluralRDD = wordsRDD.map(makePlural)

pluralRDD.collect()

['cats', 'elephants', 'rats', 'rats', 'cats']

In [30]:
# TEST Apply makePlural to the base RDD(1c)
assert pluralRDD.collect() == ['cats', 'elephants', 'rats', 'rats', 'cats'], 'incorrect values for pluralRDD'

<span style="color:navy"><strong>-> Afegeixo una celda de comprobació perquè la anterior no imprimeix cap resultat (suposo que si no diu res és correcte, però en faig una altra per a més precisó</strong></span>

In [31]:
# TEST Apply makePlural to the base RDD(1c)
expected_result = ['cats', 'elephants', 'rats', 'rats', 'cats']
actual_result = pluralRDD.collect()

if actual_result == expected_result:
    print("Test passed.")
else:
    print("Test failed. Incorrect values for pluralRDD:")
    print(f"Expected: {expected_result}")
    print(f"Actual:   {actual_result}")


Test passed.


### (1d) Executar una funció `lambda` en un` map`

Crearem el mateix RDD usant una `lambda` function en lloc d'una funció amb nom.

<span style="color:navy"><strong>-> La funció map() a Spark</strong></span>

* És una operació de transformació que aplica una funció especificada a cada element d'un RDD (Resilient Distributed Dataset) i retorna un nou RDD amb els resultats. 
* map() s'utilitza per transformar cada element d'un RDD segons una funció especificada, permetent la manipulació i el processament de dades en paral·lel a través de recursos de càlcul distribuïts.

    Entrada: Pren com a entrada un RDD.
    Funció: Aplica una funció a cada element del RDD.
    Sortida: Retorna un nou RDD que conté els resultats d'aplicar la funció a cada element.
    
<span style="color:navy"><strong>-> La funció Lambda</strong></span>

* "Lambda" és una paraula clau en Python que s'utilitza per crear funcions anònimes, petites i sense nom. La sintaxi bàsica d'una funció lambda és:
    
    *lambda arguments: expression*
    
En el nostre cas, **lambda word: word + 's'** crea una funció lambda que pren una **paraula com a argument** i després **retorna la paraula concatenada amb la lletra 's'**.

* **word** és l'argument que rep la funció lambda, que representa cada paraula en l'RDD.
* **word + 's'** és l'expressió que s'avalua per a cada paraula, que retorna la paraula original concatenada amb la lletra 's'.

<span style="color:navy"><strong>-> Per què executar una funció *lambda* en un *map*?</strong></span>
    
    Executar una funció lambda en un map és útil per aplicar una operació simple o una transformació a cada element d'un RDD sense necessitat de definir una funció separada. Les funcions lambda són breus i s'usen principalment quan només cal realitzar una operació senzilla, com ara una concatenació o una operació matemàtica, en cada element del RDD. Això fa que el codi sigui més concís i llegible anb una funció breu i senzilla.
    


In [32]:
# TODO: Replace <FILL IN> with appropriate code
# pluralLambdaRDD = wordsRDD.map(<FILL IN>)

pluralLambdaRDD = wordsRDD.map(lambda word: word + 's')
print(pluralLambdaRDD.collect())

['cats', 'elephants', 'rats', 'rats', 'cats']


El resultat és el conjunt  de paraules en plural, amb l'adició de la lletre 's'

<span style="color:navy"><strong>-> Sobre aquest codi:</strong></span></strong></span>

* Fem servir la transformació map() en l'RDD wordsRDD.
* Dins la transformació map(), definim una funció lambda que pren una paraula com a entrada i retorna la paraula amb una 's' afegida.
* La funció lambda s'aplica a cada element de l'RDD.
* Finalment, recollim els elements de l'RDD pluralLambdaRDD i els imprimim.


In [33]:
# TEST Pass a lambda function to map (1d)
assert pluralLambdaRDD.collect() == ['cats', 'elephants', 'rats', 'rats', 'cats'], 'incorrect values for pluralLambdaRDD (1d)'

# assert
'''quan l'expressió 'assert' s'avalua com a certa, no es produeix cap sortida i el programa continua amb l'execució.
No obstant això, si la condició especificada en l'assert s'avalua com a Fals, es genera un AssertionError'''

Acabem de veure que  el codi es correcte.

### (1e) Nombre de caràcters de cadascuna de les paraules

Ara farem servir un `map()` i una funció lambda `lambda` per obtenir el nombre de caràcters de cada paraula. Farem servir `collect` per guardar aquest resultat directament en una variable.

<span style="color:navy"><strong>-> Reemplacem <'FILL IN'> amb una concatenació de la variable definida a la part 1d), de .map i nchars</strong></span>

In [42]:
# nchars és el nom que li donarem a la variable que ens dirà el nombre de caràcters de cadascuna de les paraules que li demanem
nchars = lambda w : len(w)

In [43]:
# TODO: Replace <FILL IN> with appropriate code
# pluralLengths = (<FILL IN>)

pluralLengths = (pluralLambdaRDD.map(nchars).collect())
print(pluralLengths)

[4, 9, 4, 4, 4]


Acaba de donar-nos el nombre de lletres que té cada paraula del conjunt

<span style="color:navy"><strong>-> Sobre aquest codi:</strong></span></strong></span>

* La funció lambda nchars pren una paraula com a argument w i retorna la longitud (nombre de caràcters) d'aquesta paraula utilitzant la funció len().
* Llavors, utilitzem el map() en l'RDD pluralLambdaRDD per aplicar la funció nchars a cada paraula i obtenir una nova RDD amb les longituds de les paraules.
* Finalment, utilitzem collect() per obtenir una llista de totes les longituds de paraules i la guardem en la variable pluralLengths, que imprimim posteriorment.

In [44]:
# TEST Length of each word (1e)
assert pluralLengths == [4, 9, 4, 4, 4], 'incorrect values for pluralLengths'

# assert
'''quan l'expressió 'assert' s'avalua com a certa, no es produeix cap sortida i el programa continua amb l'execució.
No obstant això, si la condició especificada en l'assert s'avalua com a Fals, es genera un AssertionError'''

Acabem de veure que  el codi es correcte.

### (1f) Pair RDDs

El següent pas per a completar el nostre programa de comptatge de paraules és crear un nou tipus de RDD, anomenat pair RDD. 

Un pair RDD és un RDD on cada element és un tupla de l'estil `(k, v)` on `k` és la clau i `v` és el seu valor corresponent. 

En aquest exemple, crearem una pair RDD consistent en tuples amb el format `('<word>', 1)` per a cada element del nostre RDD bàsic.

Podem crear el nostre pair RDD usant una transformació `map()` amb una `lambda()` funció que creï un nou RDD.

* La clau 'word' o 'k' és l'element important que utilitzarem per al comptatge i l'anàlisi de les paraules.
* El valor 'v' associat a cada paraula en el pair RDD indica que cada paraula apareix v vegades.

In [46]:
# TODO: Replace <FILL IN> with appropriate code
# wordPairs = wordsRDD.map(<FILL IN>)

wordPairs = wordsRDD.map(lambda word: (word, 1))
print(wordPairs.collect())

[('cat', 1), ('elephant', 1), ('rat', 1), ('rat', 1), ('cat', 1)]


En aquest context, dins de **'wordPairs'**, el valor 1 associat a cada valor en el pair RDD indica que cada paraula apareix una vegada
########################################################

<span style="color:navy"><strong>-> Sobre aquest codi:</strong></span></strong></span>

* Utilitzem la transformació map() en l'RDD wordsRDD.
* Dins la funció lambda, creem una tupla amb la paraula com a primer element i el valor 1 com a segon element.
* La funció lambda s'aplica a cada element de l'RDD, creant així un pair RDD amb tuples ('<word>', 1).
* Finalment, recollim els elements del pair RDD wordPairs i els imprimim.

In [50]:
# TEST Pair RDDs (1f)
assert wordPairs.collect() == [('cat', 1), ('elephant', 1), ('rat', 1), ('rat', 1), ('cat', 1)], 'incorrect value for wordPairs'

## Part 2: Comptar paraules usant un pair RDD

Ara, comptarem el nombre de vegades que una paraula en particular apareix al RDD. Aquesta operació es pot realitzar d'una infinitat de maneres, però algunes seran molt menys eficients que d'altres.

Un solucio molt senzilla seriosa utilitzar `collect()` sobre tots els elements retornar-los al driver i alli comptar-los. Mentre aquesta forma de treballar podria funcionar amb textos relativament curts, nosaltres el que volem és poder treballar amb textos de qualsevol longitud. Addicionalment, executar tot el càlcul al controlador és molt més lent que executar en paral·lel en els workers. Per aquests motius, en aquesta practica farem servir operacions paralelizables.

### (2a) Usant `groupByKey()`
Una primera solució al nostre problema, després veurem que hi ha altres molt més eficients, es podria basar en la transformació [groupByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.groupByKey). Com el seu nom indica, la transformació `groupByKey ()` agrupa tots els elements d'un RDD que comparteixin la mateixa clau en una única llista dins d'una de les particions.

Aquesta operació planteja dos problemes:
  + Aquesta operació necessita moure tots els valors dins de la partició adequada. Això satura la xarxa.
  + Les llistes generades poden arribar a ser molt grans arribant fins i tot a saturar la memòria d'algun dels trabajadadores
  
Utilitza `groupByKey()` per generar un pair RDD del tipus `('word', Iterator)`.

In [54]:
# TODO: Replace <FILL IN> with appropriate code
# Note that groupByKey requires no parameters
# wordGrouped = <FILL IN>

wordsGrouped = wordPairs.groupByKey()
for key, value in wordsGrouped.collect():
    print('{0}: {1}'.format(key, list(value)))

cat: [1, 1]
elephant: [1]
rat: [1, 1]


<span style="color:navy"><strong>-> En aquest resultat:</strong></span></strong></span> 

* El resultat és la impressió que s'està fent en el bucle 'for' després d'aplicar la transformació groupByKey() al pair RDD wordPairs.
* La key/clau (paraula) 'elephant' surt diferent és perquè només hi ha una instància de la paraula elephant al text, per tant, la seva llista associada només conté un valor 1. En canvi, les altres paraules (cat i rat) apareixen dues vegades cadascuna, així que les seves llistes associades contenen dos valors
* Recordem que wordsList = ['cat', 'elephant', 'rat', 'rat', 'cat'] ---> "[('cat', 1), ('elephant', 1), ('rat', 1), ('rat', 1), ('cat', 1)]"


In [55]:
# TEST groupByKey() approach (2a)
assert sorted(wordsGrouped.mapValues(lambda x: list(x)).collect()) == [('cat', [1, 1]), ('elephant', [1]), ('rat', [1, 1])], 'incorrect value for wordsGrouped'

### (2b) Utilitza `groupByKey()` per obtenir els recomptes

Usant la transformació `groupByKey()` crea un RDD que contingui 2 elements, on cada un d'ells sigui un parell paraula (clau) Iterador de Python (valor).

Després suma tots els valors de iterator usant una transformació `map()`. El resultat ha de ser un pair RDD que contingui les parelles (word, count).

<span style="color:navy"><strong>-> Utilitzem la transformació map() en el RDD wordsGrouped, que ja ha estat agrupat per clau</strong></span></strong></span> 
* La funció lambda s'aplica a cada element del RDD wordsGrouped, que consisteix en una tupla (word, Iterator).
* La funció lambda suma tots els valors de l'iterador associat a cada paraula.
* La sortida de la funció lambda és una nova tupla amb la paraula com a clau i la suma dels valors com a recompte.

In [56]:
# TODO: Replace <FILL IN> with appropriate code
# wordCountsGrouped = <FILL IN>

wordCountsGrouped = wordsGrouped.map(lambda x: (x[0], sum(x[1])))
print(wordCountsGrouped.collect())

[('cat', 2), ('elephant', 1), ('rat', 2)]


Col·leccionem els resultats utilitzant collect() per visualitzar-los.

In [58]:
# TEST Use groupByKey() to obtain the counts (2b)
assert sorted(wordCountsGrouped.collect())==[('cat', 2), ('elephant', 1), ('rat', 2)],'incorrect value for wordCountsGrouped'


### (2c) Conteig usant `reduceByKey`

Una millor solució és començar des d'un pair RDD i aleshores utilitzar la transformació [reduceByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.reduceByKey) per crear un nou pair RDD. La transformació `reduceByKey()` agrupa totes les parelles que comparteixen la mateixa clau. Posteriorment s'aplica la funció que se li passa per paràmetre agrupant els valors de dos en dos. Aquest procés es repeteix iterativament fins que obtenim un únic valor afegit per a cadascuna de les claus de l'pair RDD. `ReduceByKey()` opera aplicant la funció primer dins de cadascuna de les particions de forma independent, i posteriorment únicament comparteix els valors agregats entre particions diferents, permetent escalar de forma eficient ja que no té necessitat de desplaçar per la xarxa una gran quantitat de dades.

<span style="color:navy"><strong>-> Combinem reduceByKey()i Lambda</strong></span></strong></span> 

* La transformació reduceByKey() agrupa totes les parelles que comparteixen la mateixa clau (paraula).
* Després, aplica una funció lambda que suma els valors associats a cada paraula.
* El resultat és un nou pair RDD que conté les parelles (word, count) on word és la paraula i count és el recompte total d'aquesta paraula al text.

In [62]:
# TODO: Replace <FILL IN> with appropriate code
# Note that reduceByKey takes in a function that accepts two values and returns a single value
# wordCounts = <FILL IN>

wordCounts = wordPairs.reduceByKey(lambda x, y: x + y)
print (wordCounts.collect())

[('cat', 2), ('elephant', 1), ('rat', 2)]


In [69]:
# TEST Counting using reduceByKey (2c)
assert sorted(wordCounts.collect())==[('cat', 2), ('elephant', 1), ('rat', 2)],'incorrect value for wordCounts'

### (2d) Ara tot junt

La versió més complexa del codi executa primer un `map()` sobre el pair RDD, la transformació `reduceByKey()`, i finalment l'acció `collect()` en una única línia de codi.

In [ ]:
wordCountsCollected = <FILL IN>
print(wordCountsCollected)

In [67]:
wordCountsCollected = wordsRDD.map(lambda word: (word, 1)).reduceByKey(lambda x, y: x + y)
# Imprimeix el resultat
print(wordCountsCollected.collect())

[('cat', 2), ('elephant', 1), ('rat', 2)]


In [64]:
# TODO: Replace <FILL IN> with appropriate code
# wordCountsCollected = <FILL IN>

# Combina map() i reduceByKey() en una sola línia
wordCountsCollected = wordsRDD.map(lambda word: (word, 1)).reduceByKey(lambda x, y: x + y)

# Imprimeix el resultat
print(wordCountsCollected.collect())

[('cat', 2), ('elephant', 1), ('rat', 2)]


<span style="color:navy"><strong>-> En aquest codi:</strong></span></strong></span> 

* La funció lambda dins de map() crea un pair RDD on cada paraula és una clau i té un valor inicial de 1. 
* Utilitzem reduceByKey() per sumar els valors per a cada clau (paraula). 
* El resultat és el pair RDD wordCountsCollected que conté les parelles (word, count) on word és la paraula i count és el recompte total d'aquesta paraula al text.

In [71]:
# TEST All together (2d)
assert sorted(wordCountsCollected)==[('cat', 2), ('elephant', 1), ('rat', 2)],'incorrect value for wordCountsCollected'

TypeError: 'PipelinedRDD' object is not iterable

<span style="color:navy"><strong>-> Comprobació i 'pipeline':</strong></span></strong></span> 

El terme **PipelinedRDD** fa referència al **tipus d'RDD que es retorna després de les transformacions** com map(), reduceByKey(), i altres en Spark. 

És un RDD que representa una seqüència de transformacions que s'aplicaran a les dades quan s'activi una acció : quan apliquem transformacions com **map()** i **reduceByKey()** a un RDD, **Spark no executa immediatament aquestes transformacions**. En lloc d'això, crea un pla de transformacions, que forma un **gràfic acíclic dirigit (DAG) que representa el càlcul a realitzar**. Aquest pla s'anomena pipeline, i s'optimitza i s'executa de manera diferida quan s'activa una acció com collect() o count().

* Quan fem el testing, surt un **TypeError** donat que 'wordCountsCollected' és **un objecte tipus 'PipelinedRDD' i no es pot sortejar directament**.
* Per resoldre això, es poden recolectar els resultats com a **llista** i després sortejar-los amb **'sorted'**, aixì : 

In [70]:
# Collect the results as a list and sort it
wordCountsList = wordCountsCollected.collect()
wordCountsSorted = sorted(wordCountsList)

# Print the sorted result
print(wordCountsSorted)

[('cat', 2), ('elephant', 1), ('rat', 2)]


Veiem que els resultats corresponen al esperats.

## Part 3: Trobar les paraules individuals i la seva freqüència d'aparició mitjana

### (3a) Paraules úniques

Calcular el nombre de paraules úniques en `wordsRDD`. Pots utitlziar altres RDDs que hagis creat en aquesta practica si et resulta més senzill.

<span style="color:navy"><strong>-> Recordem els inputs i definicions de variables de la celda [23]:</strong></span></strong></span> 

    wordsList = ['cat', 'elephant', 'rat', 'rat', 'cat']
    wordsRDD = sc.parallelize(wordsList)

In [92]:
# TODO: Replace <FILL IN> with appropriate code
# uniqueWords = <FILL IN>

uniqueWords = wordsRDD.distinct().count()
uniqueWords

3

In [93]:
# TEST Unique words (3a)
assert uniqueWords== 3, 'incorrect count of uniqueWords'

In [94]:
uniqueWordsList = wordsRDD.distinct().collect()
print(uniqueWordsList)

['cat', 'elephant', 'rat']


Veiem que els resultats corresponen al esperats i que obtenim efectivament 3 paraules diferents, que son 'cat', 'elephant', 'rat'.

### (3b) Calular la mitjana usant `reduce()`

Troba la freqüència mitjana de aparició de paraules en `wordCounts`.

Utilitza l'acció `reduce()` per sumar els recomptes en `wordCounts` i llavors divideix pel nombre de paraules úniques. Per realitzar això primer s'aplica un `map()` a pair RDD `wordCounts`, que està format per tuples amb el format (key, value), per convertir-lo en un RDD de valors.

<span style="color:navy"><strong>-> Recordem els inputs i definicions de variables de la celda [62]:</strong></span></strong></span> 

    wordCounts = wordPairs.reduceByKey(lambda x, y: x + y)
    print (wordCounts.collect())
    [('cat', 2), ('elephant', 1), ('rat', 2)]

<span style="color:navy"><strong>-> Solució:</strong></span></strong></span> 

* Per calcular la **freqüència mitjana de les paraules en wordCounts**, primer necessitem **sumar tots els recomptes** i després dividir aquesta suma pel nombre total de paraules úniques.
* Podem fer-ho utilitzant l'acció **reduce() per sumar** els valors de wordCounts i llavors **dividint pel nombre total de paraules úniques**.

In [96]:
# TODO: Replace <FILL IN> with appropriate code
# totalCount = <FILL IN>
# average = <FILL IN>

from operator import add

# Suma tots els valors de wordCounts
totalCount = wordCounts.map(lambda x: x[1]).reduce(add)

# Divideix la suma pel nombre total de paraules úniques
average = totalCount / uniqueWords

print(totalCount)
print(round(average, 2))

5
1.67


In [97]:
# TEST Mean using reduce (3b)
assert round(average, 2)==1.67, 'incorrect value of average'

Els resultats corresponen als esperats: com que tenim 5 paraules, de les quals només 3 son dferents, ens dona una mitjana d'1.67.

## Part 4: Aplicar les funcionalitats desenvolupades a un arxiu de text

Per això hem de construir una funció `wordCount`, capaç de treballar amb dades del món real que solen presenten problemes com l'ús de majúscules o minúscules, puntuació, accents, etc. Posteriorment, carregar les dades de la nostra font de dades i finalment, calular el recompte de paraules sobre les dades processades.

### (4a) funcio `wordCount`

Primer, defineix una funció per al recompte de paraules. Hauries de reutilitzar les tècniques que has vist en els apartats anteriors d'aquesta practica. Aquesta funció, ha de prendre un RDD que contingui una llista de paraules, i tornar un pair RDD que contingui totes les paraules amb els seus corresponents recomptes.

In [100]:
# TODO: Replace <FILL IN> with appropriate code

# La diferència entre 'wordCounts' i 'wordCount' és que:
# 'wordCounts' és el nom d'una variable que emmagatzema un RDD que conté el recompte de paraules
# 'wordCount' és el nom de la funció que calcula aquest recompte. 

def wordCount(wordListRDD):
    """Creates a pair RDD with word counts from an RDD of words.

    Args:
        wordListRDD (RDD of str): An RDD consisting of words.

    Returns:
        RDD of (str, int): An RDD consisting of (word, count) tuples.
    """
    
    # Compta les aparicions de cada paraula
    wordCounts = wordListRDD.map(lambda word: (word, 1)).reduceByKey(lambda x, y: x + y)
    
    # return <FILL IN>
    return wordCounts

print(wordCount(wordsRDD).collect())

[('cat', 2), ('elephant', 1), ('rat', 2)]


In [101]:
# TEST wordCount function (4a)
assert sorted(wordCount(wordsRDD).collect())==[('cat', 2), ('elephant', 1), ('rat', 2)],'incorrect definition for wordCount function'

<span style="color:navy"><strong>-> Solució i resultat:</strong></span></strong></span> 

* Aquesta funció crea un pair RDD amb el recompte de paraules a partir d'un RDD de paraules. 
* Utilitza **map()** per assignar un **valor inicial d'1** a cada paraula, després **reduceByKey()** per sumar els valors de cada paraula
* El resultat tornat és el esperat ; és un **pair RDD** que conté totes les paraules amb els seus corresponents recomptes com a **tuples (paraula, recompte)**.

### (4b) majúscules i puntuació

Els fitxers del món real són molt més complexos que els que hem estat usant en aquesta PAC. Alguns dels problemes que són necessaris de solucionar són:
  + Les paraules han de comptar-se independentment tan si estan en mayuscula o minúscula (per exemple, Spark i spark haurien explicar-se com la mateixa paraula).
  + Tots els signes de puntuació han d'eliminar-se.
  + Qualsevol espai al principi o al final de la paraula ha de eliminar-se.
  
Defineix la funció `removePunctuation` que converteixi tot el text a minúscules, elimini els signes de puntuació, i elimini els espais al principi i final de cada paraula. Utilitza el mòdul de Python [re](https://docs.python.org/2/library/re.html) per eliminar qualsevol caràcter que no sigui una lletra, un nombre o un espai.

Sinó aquestes familiaritzat amb les expressions regulars hauries revisar [aquest tutorial](https://developers.google.com/edu/python/regular-expressions). Alternativament, [aquest web](https://regex101.com/#python) és de gran ajuda per a debugar les teves expressions regulars.

** Hints **

1. Fes servir la funcio [re.sub()](https://docs.python.org/2.7/library/re.html#re.sub).
2. Per als nostres propòsits, "puntuació" significa "no alphabetico, numèric, o espai." La expressio regular que defineix aquests caràcters és: `[^A-Za-z\s\d]`
3. No fer servir `\W`, ja que retindrà els guions baixos.

In [103]:
# TODO: Replace <FILL IN> with appropriate code
import re
def removePunctuation(text):
    """Removes punctuation, changes to lower case, and strips leading and trailing spaces.

    Note:
        Only whitespace, letters, and numbers should be retained.  Other characters should should be
        eliminated (e.g. it's becomes its).  Leading and trailing spaces should be removed after
        punctuation is removed.

    Args:
        text (str): A string.

    Returns:
        str: The cleaned up string.
    """
    # Eliminem la puntuació i convertim el text a minúscules
    text = re.sub('[^A-Za-z\s\d]', '', text.lower())
    
    # Eliminem els espais al principi i al final de cada paraula
    text = text.strip()
    
    return text
    
print(removePunctuation('Hi, you!'))
print(removePunctuation(' No under_score!'))
print(removePunctuation(' *      Remove punctuation then spaces  * '))
print(removePunctuation(" The Elephant's 4 cats. "))

hi you
no underscore
remove punctuation then spaces
the elephants 4 cats


In [ ]:
# TEST Capitalization and punctuation (4b)
assert removePunctuation(" The Elephant's 4 cats. ") == 'the elephants 4 cats', 'incorrect definition for removePunctuation function'

<span style="color:navy"><strong>-> Solució i resultat:</strong></span></strong></span> 

* A la funció **removePunctuation**, el paràmetre ***text*** representa la cadena d'entrada que volem processar. 
* Quan cridem la funció removePunctuation(*'Hi, you!'*), la cadena 'Hola, tu!' és passada com a argument al paràmetre text.
* En l'expressió **return**, ***text*** fa referència a la cadena processada després d'haver eliminat la puntuació, convertit tot a minúscules i eliminat els espais al principi i al final. És la **versió modificada de la cadena (string) d'entrada** que la funció produeix com a sortida.

### (4c) Carregar un fitxer de text

Per a la següent part, farem servir el text ja esmentat ***Lorem Ipsum*** generat per a la pràctica. Para convertir un fitxer de text en un RDD, farem servir el mètode `SparkContext.textFile()`. També farem servir la funció que acabem de crear `removePunctuation()` dins d'una transformació `map()` per eliminar tots els caràcters no alphabeticos, numèrics or espais. Atès que el fitxer és bastant gran, farem servir `take(15)`, de manera que tan sols imprimirem per pantalla les 15 primeres línies.

 <span style="color:navy"><strong>-> Solució i resultat:</strong></span></strong></span> 

* El módulo os.path, que proporciona funciones para manipular rutas de archivo
* Se **leerá el contenido del archivo** de texto 'LoremIpsum.txt' en un RDD utilizando **sc.textFile()**. 
* El argumento opcional, minPartitions, especifica el número mínimo de particiones a crear al leer el archivo. En este caso, se establece en 8.
* La función **removePunctuation** a cada línea del RDD con **map()** es para eliminar la puntuación y otros caracteres no deseados
* El método **take(n)** se utiliza para **devolver una lista** con los primeros n elementos de un RDD. 
* Excluye las líneas que tienen longitud 0 (**líneas vacías**) utilizando filter().
* loremRDD.take(10) devolverá una lista con **las primeras 10 frases** del archivo loremRDD. 

In [105]:
# Tan solo ejecuta este codigo
import os.path

# Definir la ruta completa
folder_path = "C:\\Users\\Buba\\Documents\\CURSOS-PROGRAMACION\\IT-Academy\\IT-BigData\\BigData_Sprint3_ApacheSpark-PySpark"
file = "LoremIpsum.txt"

# fileName = os.path.join('/ruta/a/la/carpeta', 'LoremIpsum.txt')
# Combinar la ruta y el nombre del archivo
fileName = os.path.join(folder_path, file)

loremRDD = sc.textFile(fileName, 8).map(removePunctuation).filter(lambda x: len(x)>0)
loremRDD.take(10)

['lorem ipsum dolor sit amet consectetuer adipiscing elit sed diam nonummy nibh euismod tincidunt ut laoreet dolore magna aliquam erat volutpat ut wisi enim ad minim veniam quis nostrud exerci tation ullamcorper suscipit lobortis nisl ut aliquip ex ea commodo consequat duis autem vel eum iriure dolor in hendrerit in vulputate velit esse molestie consequat vel illum dolore eu feugiat nulla facilisis at vero eros et accumsan et iusto odio dignissim qui blandit praesent luptatum zzril delenit augue duis dolore te feugait nulla facilisi',
 'expetenda tincidunt in sed ex partem placerat sea porro commodo ex eam his putant aeterno interesset at usu ea mundi tincidunt omnium virtute aliquando ius ex ea aperiri sententiae duo usu nullam dolorum quaestio ei sit vidit facilisis ea per ne impedit iracundia neglegentur consetetur neglegentur eum ut vis animal legimus inimicus id',
 'his audiam deserunt in eum ubique voluptatibus te in reque dicta usu ne rebum dissentiet eam vim omnis deseruisse id

### (4d) Extreure les paraules de les línies

Abans de poder utilitzar la funció `wordcount()`, hem de solucionar dos problemes amb el format del RDD:
  + El primer problema és que necessitem dividir cada línia pels seus espais. **Això ho solucionarem en l'apartat (4d).**
  + El segon problema és que necessitem filtar les línies completament buides. **Això ho solucionarem en l'apartat (4e).**

Per aplicar una transformació que divideixi cada element del RDD pels seus espais, hem d'aplicar la funció incorporada en els strings de Python [split()](https://docs.python.org/2/library/string.html#string .split). Compte que a primera vista pot semblar que la funció necessària és una transformació `map()`, però si penses una mica mes sobre el resultat de la funció `split()` t'adonaràs que aquesta no és l'opció correcta.

> Nota:
> * No facis servir la implementació estàndard de l' `split()`, passa-li un valor de separació. Per exemple, per dividir `line` per comes, usa `line.split(',')`.

In [124]:
# TODO: Replace <FILL IN> with appropriate code
# loremWordsRDD = loremRDD.flatMap(<FILL IN>)
# loremWordsCount = <FILL IN>

loremWordsRDD = loremRDD.flatMap(lambda line: line.split(' ')) 
# (' ') signifca 'espacio'

loremWordsCount = loremWordsRDD.count()

loremWordsCount

2342

<span style="color:navy"><strong>-> Aplanar" con flatMap() significa convertir una estructura de datos de varias capas en una sola lista plana</strong></span></strong></span> 

* Cuando se aplica flatMap() a un RDD, se aplica una función a cada elemento del RDD y el resultado es una lista de elementos
* Luego flatMap() "aplanará" esta lista de listas en una sola lista, en lugar de una lista de listas.


In [126]:
# TEST Words from lines (4d)
# This test allows for leading spaces to be removed either before or after punctuation is removed.

assert loremWordsCount == 2342, 'incorrect value for loremWordsCount'
assert loremWordsRDD.top(5)==[u'zzril', u'zzril', u'zzril', u'zzril', u'zzril'], 'incorrect value for loremWordsRDD'

<span style="color:navy"><strong>-> Resultats i explicacions</strong></span></strong></span> 

* El resultat del word count ha anat segons esperat : 2342.
* La llista [u'zzril', u'zzril', u'zzril', u'zzril', u'zzril'] que és una llista de cadenes de caràcters. 
* El prefix u abans de cada cadena indica que són cadenes Unicode a Python 2. A Python 3, aquest prefix no està present ja que totes les cadenes són Unicode per defecte.
* Veiem la mateixa paraula repetida a la llista perquè loremWordsRDD.top(5) està recuperant les 5 primeres paraules basant-se en el seu ordre lexicogràfic, i en el text de Lorem Ipsum, "zzril" es una paraula que apareix freqüentment.

### (4e) Calcula paraules diferents

El següent pas és comptar quantes paraules diferents conté el nostre text. Podeu fer servir transformacions `map()` i `reduceByKey()` ja utilitzades anteriorment.

In [129]:
# Dividir les línies en paraules i eliminar les buides
loremWordsRDD = loremRDD.flatMap(lambda line: line.split(' ')).filter(lambda word: word != '')

# Obtenir el nombre de paraules úniques
uniqueWordCount = loremWordsRDD.distinct().count()
uniqueWordCount

364

In [135]:
# TODO: Replace <FILL IN> with appropriate code
# distintWordsMapRDD = loremWordsRDD.<FILL IN>
# distintWordsRDD=distintWordsMapRDD.<FILL IN>

# Crea un pair RDD amb totes les paraules com a clau i un valor constant
distintWordsMapRDD = loremWordsRDD.map(lambda word: (word, 1))

# Utilitza reduceByKey per eliminar les paraules duplicades
distintWordsRDD = distintWordsMapRDD.reduceByKey(lambda x, y: x)

# Obtenir només les claus, que són les paraules úniques
distintWordsRDD = distintWordsRDD.keys()

# Veure els resultats
print(distintWordsRDD.collect())


['ipsum', 'diam', 'nonummy', 'aliquip', 'illum', 'nulla', 'luptatum', 'porro', 'impedit', 'iracundia', 'reque', 'rebum', 'quaeque', 'disputando', 'homero', 'solum', 'postulant', 'an', 'accusata', 'eius', 'impetus', 'vidisse', 'apeirian', 'labores', 'persequeris', 'nemore', 'fierent', 'mollis', 'vitae', 'expetendis', 'debitis', 'delicata', 'veri', 'admodum', 'oportere', 'detraxit', 'splendide', 'alii', 'mediocrem', 'fugit', 'modo', 'congue', 'malorum', 'consectetuer', 'euismod', 'ut', 'laoreet', 'wisi', 'minim', 'autem', 'velit', 'qui', 'expetenda', 'placerat', 'aeterno', 'nullam', 'id', 'paulo', 'ridens', 'hinc', 'verear', 'quem', 'unum', 'pro', 'conclusionemque', 'malis', 'has', 'harum', 'qualisque', 'mea', 'adversarium', 'vix', 'iudicabit', 'meliore', 'fabellas', 'salutatus', 'temporibus', 'albucius', 'lucilius', 'populo', 'mutat', 'tempor', 'nihil', 'oblique', 'reprehendunt', 'repudiare', 'dicit', 'everti', 'amet', 'molestie', 'at', 'vero', 'iusto', 'odio', 'feugait', 'interesset', 

In [138]:
print(distintWordsRDD.count())

364


In [139]:
# TEST Remove empty elements (4e)
assert distintWordsRDD.count()== 364, 'incorrect value for shakeWordCount'

El número de paraules úniques és el que s'esperava (364)

### (4f) Compte les paraules

Ara que tenim un RDD que conté només paraules. El següent pas és aplicar la funció `wordCount()` per a produir una llista amb els recomptes de paraules. Podem veure les 15 més comuns usant l'acció `takeOrdered()`; però, com els elements del RRD són parells, necessitem una funció especial que ordeni els parells de la forma correcta.

Utilitza les funcions `wordCount()` i `takeOrdered()` per obtenir les 15 paraules més comuns juntament amb els seus recomptes.

<span style="color:navy"><strong>-> Per obtenir les 15 paraules més comunes juntament amb els seus recomptes:</strong></span></strong></span> 


* Aplicar la funció wordCount() sobre l'RDD loremWordsRDD per obtenir un pair RDD amb les paraules i els seus recomptes.
* Utilitzar l'acció takeOrdered() per obtenir les 15 paraules més comunes juntament amb els seus recomptes.
* La línia de codi "*top15WordsAndCounts...*" obtindrà les 15 paraules més comunes i els seus recomptes. 
* L'argument key=lambda x: -x[1] indica que volem ordenar el RDD en ordre descendent basant-nos en el segon element de cada parell, que és el recompte de paraules.

In [146]:
# TODO: Replace <FILL IN> with appropriate code
# top15WordsAndCounts = <FILL IN>

# Aplicar la funció wordCount() per obtenir un pair RDD amb les paraules i els seus recomptes
wordCountsRDD = wordCount(loremWordsRDD)

# Utilitzar takeOrdered() per obtenir les 15 paraules més comunes juntament amb els seus recomptes
top15WordsAndCounts = wordCountsRDD.takeOrdered(15, key=lambda x: -x[1])

top15WordsAndCounts

[('et', 49),
 ('id', 47),
 ('ei', 40),
 ('in', 40),
 ('ea', 40),
 ('ad', 39),
 ('ut', 37),
 ('his', 35),
 ('te', 35),
 ('at', 34),
 ('ex', 30),
 ('per', 30),
 ('ne', 30),
 ('quo', 26),
 ('sea', 26)]

In [142]:
# TEST Count the words (4f)
assert top15WordsAndCounts== [('et', 49), ('id', 47), ('ei', 40), 
                              ('in', 40), ('ea', 40), ('ad', 39), 
                              ('ut', 37), ('his', 35), ('te', 35), 
                              ('at', 34), ('ex', 30), ('per', 30), 
                              ('ne', 30), ('quo', 26), ('sea', 26)],'incorrect value for top15WordsAndCounts'

S'han obtngut els Words-counts correctament!

## Part 5: Calcular alguns estadístics

Usant les mateixes tècniques que has aplicat en els exercicis anteriors respon a les següents preguntes:

### (5a) Quantes paraules diferents tenen exactament dos 'o'?

In [148]:
# countWords_oo = distintWordsRDD.<FILL IN>

# Filtrar les paraules que continguin exactament dos 'o'
countWords_oo = distintWordsRDD.filter(lambda word: word.count('o') == 2).count()

# Imprimir el resultat
print(countWords_oo, "paraules diferents tenen exactament dos 'o'")

24 paraules diferents tenen exactament dos 'o'


### (5b) Quina és la paraula de nou lletres que més es repeteix? Quantes vegades apareix?

In [158]:
# words9Chars = distintWordsRDD.map(lambda x: (x, 1) if(len(x)==9) else None)

# Filtra les paraules que tenen nou lletres
words9Chars = distintWordsRDD.filter(lambda x: len(x) == 9)

# Compta la freqüència de cada paraula
wordCounts = words9Chars.map(lambda word: (word, 1)).reduceByKey(lambda x, y: x + y)

# Troba la paraula amb la freqüència màxima
mostCommonWord_9Chars = wordCounts.max(key=lambda x: x[1])

# Imprimeix la paraula i la seva freqüència
print("La paraula de 9 lletres que més es repeteix és ", mostCommonWord_9Chars)


La paraula de 9 lletres que més es repeteix és  ('iracundia', 1)


"**Iracundia**" apareix 1 cop, doncs semble ser l'única paraula de **9 lletres** present al text

### (5c) Quantes paraules diferents tenen més vocals que consonants?

In [160]:
# def moreVowels(word):
    # <FILL IN>

def moreVowels(word):
    vowels = "aeiouAEIOU"
    consonants = "bcdfghjklmnpqrstvwxyzBCDFGHJKLMNPQRSTVWXYZ"
    
    # el 1 representa cada aparició d'un caràcter a la paraula que compleix la condició especificada a l'expressió generadora.
    vowel_count = sum(1 for char in word if char in vowels)
    consonant_count = sum(1 for char in word if char in consonants)
    
    return vowel_count > consonant_count

# Filtra les paraules que tenen més vocals que consonants
wordsMV = distintWordsRDD.filter(moreVowels)

# Compte les paraules resultants
wordCountMV = wordsMV.count()

In [164]:
wordsMV.count()

61

In [169]:
print('El nombre de paraules que contenen més vocals que consonants és de', wordCountMV)

El nombre de paraules que contenen més vocals que consonants és de 61


In [170]:
# apliquem la funció creada a una paraula:
moreVowels('murcielago')

False

In [167]:
# wordsMV = distintWordsRDD.filter(moreVowels)
print(wordsMV.collect())


['aliquip', 'iracundia', 'reque', 'quaeque', 'eius', 'apeirian', 'vitae', 'alii', 'euismod', 'laoreet', 'autem', 'qui', 'aeterno', 'paulo', 'qualisque', 'mea', 'iudicabit', 'meliore', 'oblique', 'repudiare', 'iusto', 'odio', 'feugait', 'aperiri', 'oratio', 'suavitate', 'tibique', 'eu', 'ei', 'quo', 'inani', 'maiorum', 'aperiam', 'feugiat', 'sea', 'eam', 'usu', 'duo', 'saepe', 'aliquid', 'eum', 'aliquando', 'audiam', 'eos', 'eruditi', 'epicuri', 'aeque', 'equidem', 'augue', 'iuvaret', 'epicurei', 'iudico', 'molestiae', 'aliquam', 'ea', 'iriure', 'ius', 'quaestio', 'ubique', 'mei', 'audire']


**Resposta final** : Acabem de visualitzar totes les 61 paraules que contenen més vocals que consonants.